Import python libraries

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part, Tool, ChatSession, FunctionDeclaration
import vertexai.preview.generative_models as generative_models
from vertexai.preview.generative_models import grounding
import requests

Set your project variables. Change "YOUR_PROJECT_ID" to your GCP project ID.

In [ ]:
project_id = "YOU_PROJECT_ID"
location = "global"
region = "us-central1"

Instantiate the model

In [ ]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
)
chat = model.start_chat()

Define a simple prompt and send it to the model

In [ ]:
prompt = "Why is the sky blue?"

In [ ]:
response = chat.send_message(prompt)
#response
print(response.text)

Encode a file into a base64 encoded string
(this file is a public data set from Kaggle - https://www.kaggle.com/datasets/kyanyoga/sample-sales-data)

In [ ]:
encoded_file = base64.b64encode(open("sales_data_sample.json", "rb").read()).decode("utf-8")

file_content = Part.from_data(
    data=base64.b64decode(encoded_file), mime_type="text/plain"
)
# supported mime types at https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini

Add the file to the model's context window

In [ ]:
prompt = "Add this document to your context. If you are able to process it, provide a simple response that it was successfully uploaded"
gemini_results = chat.send_message([file_content, prompt])
print(gemini_results.text)

In [ ]:
prompt = "what's in that document?"
gemini_results = chat.send_message(prompt)
print(gemini_results.text)

Encode an image into a base64 encoded string

In [ ]:
encoded_file = base64.b64encode(open("stuff_on_a_shelf.jpg", "rb").read()).decode("utf-8")

image_content = Part.from_data(
    data=base64.b64decode(encoded_file), mime_type="image/jpeg"
)
# supported mime types at https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini

Add the image to the model's context window

In [ ]:
prompt = "what about this file?"
gemini_results = chat.send_message([image_content, prompt])
print(gemini_results.text)

Verify the model has access to both the file and the image

In [ ]:
prompt = "OK, how many text and image files have we added to your context window so far?"
gemini_results = chat.send_message(prompt)
print(gemini_results.text)

Define a function to make the interaction with the model cleaner from a programming perspective 

In [ ]:
def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        #role = chunk.candidates[0].content.role)
        text_response.append(chunk.text)
    #print(chat.history)
    return "".join(text_response)

Send some conversational prompts and print the output showing the roles (i.e. like a more traditional chat bot with user: and gemini:)

In [ ]:
prompt = "Hello."
print('user: ' + prompt)
print('gemini: ' + get_chat_response(chat, prompt))

In [ ]:
prompt = "Tell me more about that image."
print('user: ' + prompt)
print('gemini: ' + get_chat_response(chat, prompt))

In [ ]:
prompt = "What are all the colors in a rainbow?"
print('user: ' + prompt)
print(get_chat_response(chat, prompt))

In [ ]:
prompt = "Why does it appear when it rains?"
print('user: ' + prompt)
print(get_chat_response(chat, prompt))

Define a new function that will output the respoonse in a stream

In [ ]:
def get_chat_response_stream(chat: ChatSession, prompt: str) -> str:
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        print(chunk.text)
        #print(chunk)
        #return chunk.text
    #return "".join(text_response)

In [ ]:
prompt = "Hello."
get_chat_response_stream(chat, prompt)

In [ ]:
prompt = "What are all the colors in a rainbow?"
print('user: ' + prompt)
get_chat_response_stream(chat, prompt)

In [ ]:
prompt = "Why does it appear when it rains?"
get_chat_response_stream(chat, prompt)